# W&B Sweep — DQN
Búsqueda de hiperparámetros para el agente DQN en el ambiente Simple (CSTR).
- Método: Random Search
- Proyecto W&B: `Tesis_DQN`
- Arquitectura: Simple (CTRL únicamente)

## 1. Instalación e Imports

In [1]:
import os
import random
import numpy as np
import torch
import wandb
import sys

# Agregar path del proyecto
sys.path.append('../../')

from Environment.Simulation_Env.Reactor_CSTR import CSTRSimulator
from Environment.PIDControlEnv_simple import PIDControlEnv_Simple
from Agente.DQN.train_DQN import DQNTrainer
from Aux.Plots import SimplePlotter, print_summary

print('Imports completados')
print(f'PyTorch: {torch.__version__}')
print(f'Device: {"CUDA" if torch.cuda.is_available() else "CPU"}')

Imports completados
PyTorch: 2.10.0+cu128
Device: CUDA


## 2. Login W&B

In [2]:
!pip install wandb --quiet

In [3]:
wandb.login()

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
wandb: Currently logged in as: ve326684 (ve326684-universidad-ort-uruguay) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## 3. Configuración del Sweep

In [4]:
WANDB_TEAM    = 've326684-universidad-ort-uruguay'
WANDB_PROJECT = 'Tesis_DQN'

sweep_config = {
    'name':   'dqn_cstr_random_search',
    'method': 'random',

    'metric': {
        'name': 'eval_reward',
        'goal': 'maximize'
    },

    'parameters': {

        # ============ AMBIENTE ============
        'max_time_detector': {'values': [15, 30, 60]},
        'max_steps':         {'values': [20, 50, 100]},
        'reward_dead_band':  {'values': [0.01, 0.02, 0.05]},
        'delta_percent_ctrl':{'values': [0.1, 0.2, 0.3]},

        # Reward weights — combinaciones predefinidas
        'reward_weights_idx': {
            'values': [0, 1, 2, 3]  # índice a la lista definida en sweep_run()
        },

        # ============ CRITERIOS DE ESTABILIDAD ============
        'error_increase_tolerance': {'values': [1.2, 1.5, 2.0]},
        'max_sign_changes_ratio':   {'values': [0.1, 0.2, 0.3]},
        'max_abrupt_change_ratio':  {'values': [0.03, 0.05, 0.1]},
        'abrupt_change_threshold':  {'values': [0.2, 0.3, 0.5]},

        # ============ AGENTE DQN ============
        'hidden_dims_idx':    {'values': [0, 1, 2, 3]},  # índice a lista en sweep_run()
        'lr':                 {'values': [0.0001, 0.001, 0.01]},
        'epsilon_decay':      {'values': [0.99, 0.995, 0.999]},
        'target_update_freq': {'values': [50, 100, 200]},
        'batch_size':         {'values': [32, 64, 128]},
        'buffer_type':        {'values': ['simple', 'priority']},
        'buffer_size':        {'values': [5000, 10000, 50000]},
    }
}

sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT, entity=WANDB_TEAM)
print(f'Sweep creado: {sweep_id}')

Create sweep with ID: m3p9a8rm
Sweep URL: https://wandb.ai/ve326684-universidad-ort-uruguay/Tesis_DQN/sweeps/m3p9a8rm
Sweep creado: m3p9a8rm


## 4. Función de Entrenamiento

In [5]:
# ============ LISTAS DE OPCIONES PREDEFINIDAS ============

REWARD_WEIGHTS_OPTIONS = [
    {'error': 1.0, 'tiempo': 0.3, 'overshoot': 0.2, 'energy': 0.1},   # 0: balanceado (default)
    {'error': 2.0, 'tiempo': 0.1, 'overshoot': 0.5, 'energy': 0.1},   # 1: foco en error y overshoot
    {'error': 1.0, 'tiempo': 0.5, 'overshoot': 0.1, 'energy': 0.5},   # 2: foco en tiempo y energía
    {'error': 3.0, 'tiempo': 0.1, 'overshoot': 0.1, 'energy': 0.05},  # 3: solo error importa
]

HIDDEN_DIMS_OPTIONS = [
    (64, 32),
    (128, 64),
    (128, 128, 64),
    (256, 128, 64),
]

# ============ FIJOS PARA TODOS LOS RUNS ============
SEED             = 42
N_EPISODES       = 1000
EVAL_FREQUENCY   = 50
EARLY_STOPPING_PATIENCE   = 10
EARLY_STOPPING_MIN_DELTA_PCT = 0.01
N_MANIPULABLE_VARS = 2
MANIPULABLE_RANGES = [(300, 420), (99.5, 104)]
VAR_NAMES          = ['T (K)', 'V (m³)']
DT                 = 1.0
DEVICE             = 'cuda' if torch.cuda.is_available() else 'cpu'


def sweep_run():
    # -------- Inicializar run --------
    wandb.init()
    cfg = wandb.config

    # -------- Reproducibilidad --------
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark     = False
    wandb.config.update({'seed': SEED}, allow_val_change=True)

    # -------- Resolver índices --------
    reward_weights = REWARD_WEIGHTS_OPTIONS[cfg.reward_weights_idx]
    hidden_dims    = HIDDEN_DIMS_OPTIONS[cfg.hidden_dims_idx]

    # Loggear valores reales (no índices) para legibilidad en W&B
    wandb.config.update({
        'reward_weights': str(reward_weights),
        'hidden_dims':    str(hidden_dims),
    }, allow_val_change=True)

    # -------- Configurar CSTR --------
    cstr = CSTRSimulator(
        dt=DT,
        control_limits=(MANIPULABLE_RANGES[0], MANIPULABLE_RANGES[1])
    )

    # -------- Construir config del trainer --------
    trainer_config = {
        # === AMBIENTE ===
        'env_config': {
            'architecture':    'simple',
            'env_type':        'simulation',
            'n_manipulable_vars': N_MANIPULABLE_VARS,
            'manipulable_ranges': MANIPULABLE_RANGES,
            'manipulable_setpoints': None,  # random en cada episodio
            'dt_usuario':      DT,
            'max_steps':       cfg.max_steps,
            'max_time_detector': cfg.max_time_detector,
            'reward_dead_band':  cfg.reward_dead_band,
            'delta_percent_ctrl': cfg.delta_percent_ctrl,
            'reward_weights':  reward_weights,
            'pid_limits': [
                (0.01, 50.0),
                (0.001, 1.0),
                (0.0,   1.0)
            ],
            'agent_controller_config': {'agent_type': 'discrete'},
            'env_type_config': {
                'dt': DT,
                'control_limits': (MANIPULABLE_RANGES[0], MANIPULABLE_RANGES[1])
            },
            # Criterios de estabilidad
            'stability_config': {
                'error_increase_tolerance': cfg.error_increase_tolerance,
                'max_sign_changes_ratio':   cfg.max_sign_changes_ratio,
                'max_abrupt_change_ratio':  cfg.max_abrupt_change_ratio,
                'abrupt_change_threshold':  cfg.abrupt_change_threshold,
            },
        },

        # === AGENTE DQN ===
        'agent_ctrl_config': {
            'state_dim':          N_MANIPULABLE_VARS * 5,  # 5 features por variable
            'action_dim':         7,
            'n_vars':             N_MANIPULABLE_VARS,
            'hidden_dims':        hidden_dims,
            'lr':                 cfg.lr,
            'gamma':              0.99,
            'epsilon_start':      1.0,
            'epsilon_min':        0.01,
            'epsilon_decay':      cfg.epsilon_decay,
            'batch_size':         cfg.batch_size,
            'target_update_freq': cfg.target_update_freq,
            'buffer_type':        cfg.buffer_type,
            'buffer_size':        cfg.buffer_size,
            'device':             DEVICE,
            'seed':               SEED,
        },

        # === ENTRENAMIENTO ===
        'n_episodes':          N_EPISODES,
        'eval_frequency':      EVAL_FREQUENCY,
        'save_frequency':      9999,  # no guardar checkpoints en sweep
        'log_frequency':       50,
        'checkpoint_dir':      f'checkpoints/dqn_{wandb.run.name}',
        'early_stopping_patience':      EARLY_STOPPING_PATIENCE,
        'early_stopping_min_delta_pct': EARLY_STOPPING_MIN_DELTA_PCT,
        'use_wandb': True,
    }

    # -------- Conectar CSTR al ambiente --------
    # Se hace después de crear el trainer para acceder al proceso
    trainer = DQNTrainer(trainer_config)
    trainer.env.proceso.connect_external_process(cstr)

    # -------- Entrenar --------
    trainer.train()

    # -------- Métricas finales del run --------
    wandb.log({
        'final_eval_reward':  trainer.best_reward,
        'total_episodes':     len(trainer.episode_rewards),
        'final_epsilon':      trainer.epsilons[-1] if trainer.epsilons else 0,
        'final_reward_mean10': np.mean(trainer.episode_rewards[-10:]),
        'final_energy_mean10': np.mean(trainer.episode_energies[-10:]),
        'final_overshoot_mean10': np.mean(trainer.episode_max_overshoots[-10:]),
    })

    print(f'Run completado: {wandb.run.name}')
    wandb.finish()


## 5. Lanzar Sweep

In [ ]:
wandb.agent(sweep_id, function=sweep_run, count=30)

wandb: Agent Starting Run: 563ca4uo with config:
wandb: 	abrupt_change_threshold: 0.3
wandb: 	batch_size: 32
wandb: 	buffer_size: 5000
wandb: 	buffer_type: priority
wandb: 	delta_percent_ctrl: 0.2
wandb: 	epsilon_decay: 0.999
wandb: 	error_increase_tolerance: 1.2
wandb: 	hidden_dims_idx: 2
wandb: 	lr: 0.01
wandb: 	max_abrupt_change_ratio: 0.1
wandb: 	max_sign_changes_ratio: 0.1
wandb: 	max_steps: 50
wandb: 	max_time_detector: 30
wandb: 	reward_dead_band: 0.01
wandb: 	reward_weights_idx: 2
wandb: 	target_update_freq: 200
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.


/content/Agente/memory.py:198: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  next_states = torch.FloatTensor([e.next_state for e in batch]).to(self.device)



Episodio 0/1000
  Reward: -36.01
  Length: 50
  CTRL Loss: 147084.3216
  CTRL Epsilon: 0.9812

Episodio 50/1000
  Reward: -1.50
  Length: 50
  CTRL Loss: 6741.7268
  CTRL Epsilon: 0.0804
Evaluación: Reward promedio = -119.53
Agente guardado en: checkpoints/dqn_exalted-sweep-1/agent_ctrl_best.pt
Checkpoint guardado: best


wandb: WARNING Tried to log to step 50 that is less than the current step 51. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.



Episodio 100/1000
  Reward: -15.32
  Length: 50
  CTRL Loss: 21535.7710
  CTRL Epsilon: 0.0100
Evaluación: Reward promedio = -35.77
Agente guardado en: checkpoints/dqn_exalted-sweep-1/agent_ctrl_best.pt
Checkpoint guardado: best


...

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/wandb/agents/pyagent.py", line 304, in _run_job
    self._function()
  File "/tmp/ipython-input-8490/3235770438.py", line 144, in sweep_run
    print(f'Run completado: {wandb.run.name}')
                             ^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'name'

wandb: ERROR Run fw299qso errored: 'NoneType' object has no attribute 'name'
wandb: Agent Starting Run: baeyl2g2 with config:
wandb: 	abrupt_change_threshold: 0.3
wandb: 	batch_size: 32
wandb: 	buffer_size: 10000
wandb: 	buffer_type: simple
wandb: 	delta_percent_ctrl: 0.3
wandb: 	epsilon_decay: 0.999
wandb: 	error_increase_tolerance: 1.2
wandb: 	hidden_dims_idx: 0
wandb: 	lr: 0.01
wandb: 	max_abrupt_change_ratio: 0.05
wandb: 	max_sign_changes_ratio: 0.1
wandb: 	max_steps: 20
wandb: 	max_time_detector: 15
wandb: 	reward_dead_band: 0.02
wandb: 	reward_weights_idx: 1
wandb: 	target_update_freq: 100
wandb: [wandb.login()] L


Episodio 0/1000
  Reward: -30.88
  Length: 20
  CTRL Loss: 0.0000
  CTRL Epsilon: 1.0000

Episodio 50/1000
  Reward: -4.00
  Length: 20
  CTRL Loss: 937.0982
  CTRL Epsilon: 0.3718
Evaluación: Reward promedio = -6.85
Agente guardado en: checkpoints/dqn_whole-sweep-3/agent_ctrl_best.pt
Checkpoint guardado: best

Episodio 100/1000
  Reward: -16.36
  Length: 20
  CTRL Loss: 14894.3767
  CTRL Epsilon: 0.1367
Evaluación: Reward promedio = -17.33
  Sin mejora: 1/10

Episodio 150/1000
  Reward: -4.55
  Length: 20
  CTRL Loss: 21182.6911
  CTRL Epsilon: 0.0503
Evaluación: Reward promedio = -8.58
  Sin mejora: 2/10

Episodio 200/1000
  Reward: -11.10
  Length: 20
  CTRL Loss: 94371.7623
  CTRL Epsilon: 0.0185
Evaluación: Reward promedio = -12.59
  Sin mejora: 3/10

Episodio 250/1000
  Reward: -3.75
  Length: 20
  CTRL Loss: 198505.6420
  CTRL Epsilon: 0.0100
Evaluación: Reward promedio = -10.24
  Sin mejora: 4/10

Episodio 300/1000
  Reward: -27.32
  Length: 20
  CTRL Loss: 252624.9389
  CTRL 

energy,▇▇▇▇▇▇▇▇▇█▇▇▇▇▇█▁▇▇▇▇▇█▇▇▇█▇▇▇██▇▇▇▇▇▇▇█
epsilon,█▆▄▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_reward,█▅▇▆▇▁▃▆▂▇▇
final_energy_mean10,▁
final_epsilon,▁
final_eval_reward,▁
final_overshoot_mean10,▁
final_reward_mean10,▁
kd_var0,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
kd_var1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁███▆█▁▁▁
+8,...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/wandb/agents/pyagent.py", line 304, in _run_job
    self._function()
  File "/tmp/ipython-input-8490/3235770438.py", line 144, in sweep_run
    print(f'Run completado: {wandb.run.name}')
                             ^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'name'

wandb: ERROR Run baeyl2g2 errored: 'NoneType' object has no attribute 'name'
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: tn8pb4ae with config:
wandb: 	abrupt_change_threshold: 0.2
wandb: 	batch_size: 128
wandb: 	buffer_size: 5000
wandb: 	buffer_type: simple
wandb: 	delta_percent_ctrl: 0.3
wandb: 	epsilon_decay: 0.999
wandb: 	error_increase_tolerance: 1.2
wandb: 	hidden_dims_idx: 2
wandb: 	lr: 0.0001
wandb: 	max_abrupt_change_ratio: 0.1
wandb: 	max_sign_changes_ratio: 0.1
wandb: 	max_steps: 100
wandb: 	max_time_detector: 60
wandb: 	reward_dead_band: 0.02
wandb: 	reward_weights_idx


Episodio 0/1000
  Reward: -41.48
  Length: 100
  CTRL Loss: 0.0000
  CTRL Epsilon: 1.0000

Episodio 50/1000
  Reward: -26.62
  Length: 100
  CTRL Loss: 3046.6725
  CTRL Epsilon: 0.0100
Evaluación: Reward promedio = -53.49
Agente guardado en: checkpoints/dqn_glorious-sweep-4/agent_ctrl_best.pt
Checkpoint guardado: best

Episodio 100/1000
  Reward: -223.26
  Length: 100
  CTRL Loss: 59075.3368
  CTRL Epsilon: 0.0100
Evaluación: Reward promedio = -236.27
  Sin mejora: 1/10


## 6. Visualización local del mejor run
Ejecutar DESPUÉS de que termine el sweep.

In [ ]:
from Aux.Plots import SimplePlotter, print_summary

# Recuperar el mejor run del sweep
api = wandb.Api()
sweep = api.sweep(f'{WANDB_TEAM}/{WANDB_PROJECT}/{sweep_id}')
best_run = sorted(sweep.runs, key=lambda r: r.summary.get('eval_reward', -float('inf')), reverse=True)[0]

print(f'Mejor run: {best_run.name}')
print(f'eval_reward: {best_run.summary.get("eval_reward"):.4f}')
print(f'Hiperparámetros:')
for k, v in best_run.config.items():
    print(f'  {k}: {v}')